# Working with [Zillow's Spatio-Temporal Data](https://www.zillow.com/research/data/)

To date, we have built a variety of tools to access and work with both spatial and flat data. In most cases, however, we are concerned with time and space, so how should we use spatio-temporal data? This Notebook provides one approach to such a question, by way of freely available data from Zillo. Our goal is to explore the evolution of home values in Washington DC.

In [1]:
# Data manipulation
import numpy as np
import pandas as pd
import json
import copy
# Visualization
import bokeh.plotting as bp
import bokeh.models as bm
import bokeh.io as bio
# Interactivity
import IPython.display as ipd
import ipywidgets as ipw
# Color mapping
import matplotlib as mpl
import matplotlib.cm as cmap

# Define function to allow us to display charts in an iframe
def show_iframe(url, iheight=400, iwidth=1000):
    display_string = '<iframe src={url} width={w} height={h}></iframe>'.format(url=url, w=iwidth, h=iheight)
    print(display_string)
    return ipd.HTML(display_string)

fig_dir = '../figs/'
data_dir = '../data/'

## Data Source: [Zillow Research](https://www.zillow.com/research/)

Data on sales of real property is in the public domain. If you choose to do so, you can easily determine the sale price of virtually any residential real estate transaction in Washington DC through the [online query tool](https://www.taxpayerservicecenter.com/RP_Search.jsp?search_type=Assessment) that DC's [Office of Tax and Revenue's](https://otr.cfo.dc.gov/) makes available. The value that Zillow brings is (at least) three-fold:

1. They collect and standardize real property data across markets;
2. They track or derive a variety of metrics in an ongoing time series;
3. They provide much (if not all) of this work to the public.

In [2]:
show_iframe('https://www.zillow.com/research/', iheight=400, iwidth='100%')

<iframe src=https://www.zillow.com/research/ width=100% height=400></iframe>


Our goal is explore the dynamics of the DC real estate market, by comparing real estate activity across the neighborhoods in DC.  This market analysis is the kind of work one might want to do if they viewed a home purchase as an investment. Analogous market analyses could be performed when one is making ... I don't know, siting decisions. With any luck, this analysis can show you how you can ask questions that are often difficult to answer from existing sources of information (especially if you end seeking to buy or rent a home in DC).

Since ZIP codes often do not map all that well to neighborhoods in a real estate sense, we are going to use [Zillow neighborhoods](https://www.zillow.com/howto/api/neighborhood-boundaries.htm) as the basis for our analysis.  Our measure will be the [Zillow Home Value Index](https://www.zillow.com/research/why-zillow-home-value-index-better-17742/), which is an estimate of the median home value in a given geographic area.  It is an estimate because only a fraction of the homes in a given area are sold in a given year. In other words, price discovery is incomplete at any one time. We won't dive too far into the mechanical details of the estimate, but you should check out the [methodology](https://www.zillow.com/research/zhvi-methodology-6032/).

Normally, we would seek to use the API for data collection, but it seems as though the API is geared towards embedded content that seeks to gather information about specific properties or provide point-in-time comparisons. However, Zillow has made the spatio-temporal data we seek readily [available for download](https://www.zillow.com/research/data/). If we peek under the hood of that website (you can view the source HTML for any page with `CTRL + u`), and search for data formats (in this case `.csv`) we can see that the ZHVI data at the neighborhood level sits in the following location:

[`http://files.zillowstatic.com/research/public/Neighborhood/Neighborhood_Zhvi_Summary_AllHomes.csv`](http://files.zillowstatic.com/research/public/Neighborhood/Neighborhood_Zhvi_Summary_AllHomes.csv)
    
We will need to marry these data with geographic areas, and the shapefile that contains said areas is located here:

[`https://www.zillowstatic.com/static-neighborhood-boundaries/LATEST/static-neighborhood-boundaries/shp/ZillowNeighborhoods-DC.zip`](https://www.zillowstatic.com/static-neighborhood-boundaries/LATEST/static-neighborhood-boundaries/shp/ZillowNeighborhoods-DC.zip)
    
For this exercise, the shapefile has already been downloaded and converted to [GeoJSON](http://geojson.org/) with a [reasonable projection](http://spatialreference.org/ref/epsg/2248/) for the Mid-Atlantic region:

    ogr2ogr -f GeoJSON -t_srs EPSG:2248 ZillowNeighborhoods-DC.geojson ZillowNeighborhoods-DC.shp

## Data Read

The first step is to acquire the data and deposit it into our `data/` folder. Once again, we can rely on our handy bash tool, [`wget`](https://www.gnu.org/software/wget/).

In [3]:
!wget -O $data_dir/ZillowNeighborhoods-DC.zip https://www.zillowstatic.com/static-neighborhood-boundaries/LATEST/static-neighborhood-boundaries/shp/ZillowNeighborhoods-DC.zip
!wget -O $data_dir/Neighborhood_Zhvi_AllHomes.csv http://files.zillowstatic.com/research/public/Neighborhood/Neighborhood_Zhvi_AllHomes.csv

--2018-04-16 18:28:32--  https://www.zillowstatic.com/static-neighborhood-boundaries/LATEST/static-neighborhood-boundaries/shp/ZillowNeighborhoods-DC.zip
Resolving www.zillowstatic.com (www.zillowstatic.com)... 54.192.53.234
Connecting to www.zillowstatic.com (www.zillowstatic.com)|54.192.53.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275935 (269K) [application/zip]
Saving to: ‘../data//ZillowNeighborhoods-DC.zip’

../data//ZillowNeig 100%[===================>] 269.47K  --.-KB/s    in 0.1s    

2018-04-16 18:28:32 (1.90 MB/s) - ‘../data//ZillowNeighborhoods-DC.zip’ saved [275935/275935]

--2018-04-16 18:28:33--  http://files.zillowstatic.com/research/public/Neighborhood/Neighborhood_Zhvi_AllHomes.csv
Resolving files.zillowstatic.com (files.zillowstatic.com)... 54.230.19.254, 54.230.19.159, 54.230.19.105, ...
Connecting to files.zillowstatic.com (files.zillowstatic.com)|54.230.19.254|:80... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

We also need to unzip the data and convert it to geojson.  Note that since we want to perform these tasks in a directory that differs from our current working directory, we need to leverage the destination flag (`-d`) in our `unzip` call, followed by the location in which we want the inflated shapefile to land. Note also that we need to provide more extensive path information in the `ogr2ogr` call.

In [4]:
cwd = !pwd
!echo Current Working Directory: $cwd
!echo Destination Directory - relative to CWD: $data_dir
!unzip ZillowNeighborhoods-DC.zip -d $data_dir
!ogr2ogr -f GeoJSON -t_srs EPSG:2248 $data_dir/ZillowNeighborhoods-DC.geojson $data_dir/ZillowNeighborhoods-DC.shp

Current Working Directory: [/home/tom/telling_stories_with_data/examples/zillow/src]
Destination Directory - relative to CWD: ../data/
unzip:  cannot find or open ZillowNeighborhoods-DC.zip, ZillowNeighborhoods-DC.zip.zip or ZillowNeighborhoods-DC.zip.ZIP.
ogr2ogr: Relink `/home/tom/miniconda3/bin/../lib/libpng16.so.16' with `/lib/x86_64-linux-gnu/libpthread.so.0' for IFUNC symbol `longjmp'
FAILURE:
Unable to open datasource `../data//ZillowNeighborhoods-DC.shp' with the following drivers.
  -> `PCIDSK'
  -> `netCDF'
  -> `JP2OpenJPEG'
  -> `PDF'
  -> `ESRI Shapefile'
  -> `MapInfo File'
  -> `UK .NTF'
  -> `OGR_SDTS'
  -> `S57'
  -> `DGN'
  -> `OGR_VRT'
  -> `REC'
  -> `Memory'
  -> `BNA'
  -> `CSV'
  -> `NAS'
  -> `GML'
  -> `GPX'
  -> `LIBKML'
  -> `KML'
  -> `GeoJSON'
  -> `Interlis 1'
  -> `Interlis 2'
  -> `OGR_GMT'
  -> `GPKG'
  -> `SQLite'
  -> `OGR_DODS'
  -> `WAsP'
  -> `PostgreSQL'
  -> `OpenFileGDB'
  -> `XPlane'
  -> `DXF'
  -> `CAD'
  -> `Geoconcept'
  -> `GeoRSS'
  -> `G

We should now have our flat data in `.csv` and our shape data in `.geojson`. We can remove all other data.

In [5]:
print('Before Clean Up:')
!ls -lah $data_dir
print('\n')

for fmt in ['dbf', 'prj', 'shp', 'shx', 'zip']:
     tmp_file = data_dir + 'ZillowNeighborhoods-DC.' + fmt
     !echo rm $tmp_file
     !rm $tmp_file
    
print('\n')
print('After Clean Up:')
!ls -lah $data_dir

Before Clean Up:
total 13M
drwxr-xr-x 2 tom  tom  4.0K Apr 16 18:28 .
drwxr-xr-x 5 tom  tom  4.0K Apr 16 18:03 ..
-rw-r--r-- 1 root root 9.6M Mar 22 00:00 Neighborhood_Zhvi_AllHomes.csv
-rw-rw-r-- 1 tom  tom  1.3M Apr 16 17:58 Neighborhood_Zhvi_Summary_AllHomes.csv
-rw-r--r-- 1 root root 1.3M Apr 16 18:04 ZillowNeighborhoods-DC.geojson
-rw-r--r-- 1 root root 270K Feb 20 07:39 ZillowNeighborhoods-DC.zip


rm ../data/ZillowNeighborhoods-DC.dbf
rm: cannot remove '../data/ZillowNeighborhoods-DC.dbf': No such file or directory
rm ../data/ZillowNeighborhoods-DC.prj
rm: cannot remove '../data/ZillowNeighborhoods-DC.prj': No such file or directory
rm ../data/ZillowNeighborhoods-DC.shp
rm: cannot remove '../data/ZillowNeighborhoods-DC.shp': No such file or directory
rm ../data/ZillowNeighborhoods-DC.shx
rm: cannot remove '../data/ZillowNeighborhoods-DC.shx': No such file or directory
rm ../data/ZillowNeighborhoods-DC.zip


After Clean Up:
total 13M
drwxr-xr-x 2 tom  tom  4.0K Apr 16 18:28 .
drw

### Flat Data

The first step is get our ZHVI data in order.  We only need to retain a ZHVI value for each neighborhood in DC, and enough information to know which neighborhood is which (for inspection and joining).

In [6]:
# Read in data
zhvi = pd.read_csv(data_dir + 'Neighborhood_Zhvi_AllHomes.csv')

# Need to move things around to make it machine readable

# Define initial index variables to ease conversion of time series to long format
init_idx_vars = ['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank']
zhvi.set_index(init_idx_vars, inplace=True)

# Stack time series data
zhvi = pd.DataFrame(zhvi.stack())

# Reset the index, convert variables to lower case
zhvi = zhvi.reset_index().rename(columns=dict(zip(init_idx_vars, 
                                                  [v.lower() for v in init_idx_vars]))) 

# Rename data series (from 0 to zhvi)
zhvi.columns = [v.lower() for v in init_idx_vars] + ['month_str', 'zhvi']

# Filter to only DC neighborhoods
zhvi = zhvi[zhvi['state'] == 'DC']

# Convert month to a pandas period object (for data operations below)
## 'Pandas period object' to capture time; if we represent strings or numbers in another way, it won't be spaced or sorted correctly
## if integer, huge gap between '201812' and '201901'; helps with re-sampling, too
## makes each string match with an integer ('201801'='1'); gives us the labels (201801), but we can sort by the integer (1)
zhvi['month'] = zhvi['month_str'].apply(lambda m: pd.Period(m, freq='M'))

# Provide an integer index for months (makes it easier to plot time series with Bokeh)
time_range_in_months = list(range((zhvi['month'].max() - zhvi['month'].min()) + 1))
time_range_in_periods = sorted(set(zhvi['month']))
time_range_map = dict(zip(time_range_in_periods, time_range_in_months))
zhvi['month_int'] = zhvi['month'].map(time_range_map)

# Map month_int to month_str
month_map = dict(zip(zhvi['month_str'], zhvi['month_int']))

# Subset to relevant variables
idx_vars = ['regionid', 'regionname', 'state', 'month_str', 'month_int', 'month']
zhvi = zhvi[idx_vars + ['zhvi']].reset_index()
zhvi.pop('index')

# Map colors to each zhvi value
## 'RdBu' is one of the stock MatPlotLab palettes
zhvi_min, zhvi_max = zhvi['zhvi'].min(), zhvi['zhvi'].max()
norm = mpl.colors.Normalize(vmin=zhvi_min, vmax=zhvi_max)
colormap = cmap.ScalarMappable(norm=norm, cmap='RdBu')
zhvi['zhvi_color'] = zhvi['zhvi'].apply(lambda x: mpl.colors.to_hex(colormap.to_rgba(x)))

# Capture neighborhoods that exist in the flat data
## Neighborhoods that are in the flat data aren't necessarily the same ones referenced in the map
nhb_names = set(zhvi['regionname'])


zhvi.head()

,regionid,regionname,state,month_str,month_int,month,zhvi,zhvi_color
0,121697,Columbia Heights,DC,1999-06,12,1999-06,121700.0,#7c0722
1,121697,Columbia Heights,DC,1999-07,13,1999-07,121100.0,#7c0722
2,121697,Columbia Heights,DC,1999-08,14,1999-08,121000.0,#7c0722
3,121697,Columbia Heights,DC,1999-09,15,1999-09,121900.0,#7c0722
4,121697,Columbia Heights,DC,1999-10,16,1999-10,124400.0,#7c0722


### Spatial Data

As always, let's take a quick look at the geojson file.

In [7]:
!head -6 $data_dir/ZillowNeighborhoods-DC.geojson

# Coordinate reference system: orients where points on a map belong based on the 'datum'
## Info below shows the coordinates for Barnaby Woods

{
"type": "FeatureCollection",
"name": "ZillowNeighborhoods-DC",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:EPSG::2248" } },
"features": [
{ "type": "Feature", "properties": { "State": "DC", "County": "District of Columbia", "City": "Washington", "Name": "Barnaby Woods", "RegionID": "121672" }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 1299460.226205993676558, 476748.699325756577309 ], [ 1299415.532778077991679, 475400.906815704016481 ], [ 1298299.129284598398954, 475446.407025186752435 ], [ 1297197.131386257009581, 475718.162995011138264 ], [ 1297045.589434839552268, 475714.407645344152115 ], [ 1296736.687303835293278, 475703.812826083158143 ], [ 1296482.093089353758842, 475695.10963323537726 ], [ 1296012.548729722155258, 475691.007994434447028 ], [ 1295636.853372227633372, 475688.706646626407746 ], [ 1295449.006486361846328, 475687.506112533446867 ], [ 1294939.125634682597592, 475686.799749882717151 ], [ 1294670.798251176718622, 475690.1688489618245

After dealing with the GeoJSON file we used in the [Chicago Access](https://github.com/choct155/telling_stories_with_data/blob/master/examples/jpmci_access/src/chicago_access.ipynb) example, this should look familiar. Our task this time, however, is a little different.  The data are already reflecting only the subset of neighborhoods that we care about.  The difference this time is that we will modify the geojson string directly instead of leveraging [GeoPandas](http://geopandas.org/) to join information to each feature. [[Last time, we joined a data frame to a GeoPandas frame; a one-to-one relationship.]]

Our challenge, however, is that only one month's value can be associated with a neighborhood at a time. So, we *need to either create a new geojson representation for each month*, or *we need some way to update one instance of geojson*. The latter is far more efficient from a space perspective, and we are dealing with a small enough set to have little to fear from update processing time.  Let's read in our data.

In [8]:
with open(data_dir + 'ZillowNeighborhoods-DC.geojson', 'r') as gj:
    nhb = json.load(gj)

We can define a function that drops the relevant values (month and ZHVI info) into the `properties` section of each feature.  Note that being able to do this easily and quickly is a big reason why dictionaries are awesome. For perspective, the function we will define below is doing the work that joining pandas DataFrames to GeoDataFrames (from GeoPandas) did in the Chicago Access example.

In [9]:
def update_geojson(src_dict=nhb, month='2017-12'):
    """
    This function will take a dictionary (converted from geojson) and return
    a dictionary with features that have been updated to reflect ZHVI values
    for neighborhoods in a given month. In short, we want to add stuff to each dictionary entry.
    """
    # For each neighborhood...
    for f in src_dict['features']:
        #... update month information...
        f['properties'].update({'month_str': month})
        f['properties'].update({'month_int': int(month_map[month])})
        #...identify the neighborhood for that feature from geojson...
        tmp_nhb = f['properties']['Name']
        # Note that some neighborhoods aren't in the time series data at all, or
        # during early spells
        try:
            #...identify the corresponding zhvi/color for that month and neighborhood...
            mth_filter = (zhvi['month_str'] == month)
            nhb_filter = (zhvi['regionname'] == tmp_nhb)
            tmp_zhvi = zhvi[mth_filter & nhb_filter]['zhvi'].iloc[0]
            tmp_zhvi_color = zhvi[mth_filter & nhb_filter]['zhvi_color'].iloc[0]
            #... update zhvi/color for that feature
            f['properties'].update({'zhvi': float(tmp_zhvi)})
            f['properties'].update({'zhvi_color': tmp_zhvi_color})
        except:
            f['properties'].update({'zhvi': float(-999)})
            f['properties'].update({'zhvi_color': '#404144'})
        
    return src_dict

new_geo = update_geojson(month='2017-01')

Dictionaries are great because they are very fast. This is good if we want to do interactive charts; we don't want a lot of lag.

It will also be useful to capture just one of the neighborhoods at any given time, so that we may highlight that neighborhood. In later versions of Bokeh, there are methods that allow us to filter on a common data source. With the current version (0.12.6) no such capability exists.  Rather than just upgrade, let's take the opportunity to see how we could build a straightforward filtering function.

In [10]:
def subset_geojson(src_dict=nhb, month='2017-12', neighborhood='Stronghold'):
    """
    This function will take a dictionary (converted from geojson) and return
    a dictionary that includes only the feature of interest in the specified
    month. The color and ZHVI value reflect the month/neighborhood combination.
    We are doing this to create the yellow border around, e.g., Stronghold neighborhood.
    """
    # Make a copy of the dictionary. Note that we need a deep copy
    # HT: https://stackoverflow.com/questions/5105517/deep-copy-of-a-dict-in-python
    sub_dict = copy.deepcopy(src_dict)
    # Identify the feature list that contains only 'neighborhood'
    new_feature_list = [f for f in src_dict['features'] if f['properties']['Name'] == neighborhood]
    # Replace total feature list with subset
    sub_dict['features'] = new_feature_list
    #Identify the corresponding zhvi/color for that month and neighborhood...
    mth_filter = (zhvi['month_str'] == month)
    nhb_filter = (zhvi['regionname'] == neighborhood)
    tmp_zhvi = zhvi[mth_filter & nhb_filter]['zhvi'].iloc[0]
    tmp_zhvi_color = zhvi[mth_filter & nhb_filter]['zhvi_color'].iloc[0]
    #... update zhvi/color for that feature
    sub_dict['features'][0]['properties'].update({'zhvi': float(tmp_zhvi)})
    sub_dict['features'][0]['properties'].update({'zhvi_color': tmp_zhvi_color})
    return sub_dict
    
new_geo_sub = subset_geojson()

Let's plot some data to make sure we have our ducks in a row. We can use basically the same function as we did with Chicago.

In [11]:
# Define data source
geo_src = bm.GeoJSONDataSource(geojson=json.dumps(new_geo))

def plot_zhvi(src_dict=nhb, month='2017-01', nhb='Stronghold', height_in=900, width_in=750, save=True):
    # Update source data and define subset
    new_src_dict = update_geojson(src_dict=src_dict, month=month)
    new_src = bm.GeoJSONDataSource(geojson=json.dumps(new_src_dict))
    try:
        new_sub_dict = subset_geojson(src_dict=src_dict, month=month, neighborhood=nhb)
        new_sub = bm.GeoJSONDataSource(geojson=json.dumps(new_sub_dict))
    except:
        print('{} has no data at that time!')
        new_sub = None
    # Define location of output file
    outf = fig_dir + 'zhvi_dc.html'
    if save:
        bp.output_file(outf)
    # Define figure; whole map call is this comment, and 'add zip polygons' below; very fast!
    fig = bp.figure(title='Zillow Home Value Index: {}'.format(month),
                       height=height_in, width=width_in)
    # Add zip polygons
    fig.patches(xs='xs', ys='ys', alpha=0.9, source=new_src,
                fill_color={'field': 'zhvi_color'}, line_width=1, line_color='#e0e2e5')
    # Highlight featured neighborhood; add yellow around the specified neighborhood. make it conditional
    # because if the neighborhood didn't exist (without the conditional), it would crash.
    if new_sub != None:
        fig.patches(xs='xs', ys='ys', alpha=0.9, source=new_sub, line_width=6, line_color='#f7a72e')
    # Define hover appearance (in raw HTML)
    hover_html = '''
    <div><b>Neighborhood</b>: @Name</div>
    <div><b>ZHVI</b>: @zhvi</div>
    '''
    hover = bm.HoverTool(
        point_policy='follow_mouse',
        tooltips=(hover_html)
    )
    # Define tap tool
    tap = bm.TapTool()
    fig.add_tools(hover)
    # Add map tile
#     fig.add_tile(CARTODBPOSITRON_RETINA)
    # Get rid of axis lines
    fig.xaxis.visible = False
    fig.yaxis.visible = False
    fig.grid.visible = False
    # Show the figure
    if save:
        bp.save(fig)
    # Note that we are returning the file path (for the iframe) and the figure
    return outf, fig

zhvi_map = plot_zhvi()
show_iframe(zhvi_map[0], iwidth=850, iheight=1050)

<iframe src=../figs/zhvi_dc.html width=850 height=1050></iframe>


## Evolving Prices over Time

Now that we have this data in hand, one wonders if this picture ever changes? How much are home prices increasing over time? Is growth in one neighborhood a lot or a little? One way to get at these questions is via a simple line chart. We can view how each neighborhood's ZHVI has changed over time. Moreover, we can parametrically highlight one neighborhood while muting the others.

To do this, let's first create a DataFrame that has a ZHVI series for each neighborhood.

In [15]:
# Same data; swing the neighborhoods out (left to right); we have neighborhood-specific series.

zhvi_nhb = zhvi.set_index(['regionname', 'month_str', 'month_int'])['zhvi'].unstack('regionname')

zhvi_nhb.head()

,regionname,Adams Morgan,American University Park,Anacostia,Barnaby Woods,Barney Circle,Bellevue,Benning,Benning Heights,Benning Ridge,Berkley,...,Takoma,The Palisades,Trinidad,Truxton Circle,U Street Corridor,Wakefield,Washington Highlands,Wesley Heights,Woodley Park,Woodridge
month_str,month_int,,,,,,,,,,,,,,,,,,,,,
1998-06,0,NaN,NaN,90900.0,NaN,126900.0,88300.0,101400.0,NaN,NaN,NaN,...,NaN,NaN,NaN,105400.0,NaN,NaN,87100.0,194800.0,277500.0,NaN
1998-07,1,NaN,NaN,88200.0,NaN,126600.0,88200.0,103600.0,NaN,NaN,NaN,...,NaN,NaN,NaN,107500.0,NaN,NaN,85900.0,196400.0,280800.0,NaN
1998-08,2,NaN,NaN,87600.0,NaN,125400.0,87200.0,105100.0,NaN,NaN,NaN,...,NaN,NaN,NaN,107900.0,NaN,NaN,83800.0,195700.0,280900.0,NaN
1998-09,3,NaN,NaN,87800.0,NaN,125300.0,86600.0,105300.0,NaN,NaN,NaN,...,NaN,NaN,NaN,109000.0,NaN,NaN,83200.0,194600.0,282400.0,NaN
1998-10,4,NaN,NaN,86300.0,NaN,124800.0,86000.0,103500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,111200.0,NaN,NaN,83400.0,194100.0,283700.0,NaN


Now we want a function that can plot all of these lines at once, while still maintaining focus on the neighborhood in question.

In [16]:
def plot_zhvi_ts(month='2017-01', nhb='Stronghold', compare_nhbs=list(nhb_names), height_in=500, width_in=750, save=True):
    # Create source
    src = bm.ColumnDataSource(zhvi_nhb.reset_index())
    # If primary neighborhood isn't already split out, do so; leave out the one we want to highlight
    comp_nhbs = [n for n in nhb_names if n != nhb]
    # Define location of output file
    outf = fig_dir + 'zhvi_dc_ts.html'
    if save:
        bp.output_file(outf)
    # Define figure
    fig = bp.figure(title='Zillow Home Value Index: {}'.format(nhb),
                       height=height_in, width=width_in)
    # Define hover appearance (in raw HTML)
    hover_html = '''
    <div><b>Neighborhood</b>: {nhb}</div>
    <div><b>Month</b>: @month_str</div>
    <div><b>ZHVI</b>: @{nhb}</div>
    '''.format(nhb=nhb)
    hover = bm.HoverTool(
        point_policy='follow_mouse',
        tooltips=(hover_html)
    )
    # Define tap tool
    tap = bm.TapTool()
    fig.add_tools(hover, tap)
    # For each comparison neighborhood, plot data with muted color
    ## For every neighborhood in our list, add a line
    for n in comp_nhbs:
        fig.line(x='month_int', y=n, source=src, alpha=0.8,
                line_color='#d9d9db', line_width=0.5)
    # Plot the primary neighborhood
    ## Add a line for the primary one; different color
    fig.line(x='month_int', y=nhb, source=src, alpha=1.,
                line_color='#f7a72e', line_width=1.5)
    # Create a vertical line at the specified month
    vline = bm.Span(location=month_map[month], dimension='height', line_color='#000000', line_width=1)
    fig.renderers.extend([vline])
    # Get rid of axis lines
    fig.xaxis.visible = False
    fig.yaxis.visible = False
    fig.grid.visible = False
    # Show the figure
    if save:
        bp.save(fig)
    # Note that we are returning the file path (for the iframe) and the figure
    return outf, fig
    
zhvi_ts = plot_zhvi_ts()
show_iframe(zhvi_ts[0], iwidth=850, iheight=550)

<iframe src=../figs/zhvi_dc_ts.html width=850 height=550></iframe>


## Linking Plots Together

While Bokeh does provide support for linking plots, this is typically driven by filtering the same underlying data source. In our case, we actually have multiple data sources (`.geojson` for the map and a flat `.csv` for the time series). Consequently, our filtering strategy is going to rely on leveraging [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/) to identify common filter keys.  `ipywidgets` are a very handy feature of the Jupyter ecosystem, insofar as they allow custom GUI assets for interactive exploration.

For our purposes, we want to update the month and neighborhood, and then have our plots respond accordingly. So what we need to do is generate 1) a dropdown list to enable neighborhood selection and 2) a slider to enable movement along the time dimension.  Here are the widgets as standalone objects.

No default histogram; instead, must make a quadrilateral plot. Need to define the rectangles; use NUMPY's histogram method to define how high to draw the rectangles, etc.

In [28]:
# Generate widgets
tmp_dropdown = ipw.widgets.Select(
    options=nhb_names,
    value='Stronghold',
    rows=8,
    description='Neighborhood:',
    disabled=False,
    style = {'description_width': 'initial'}
)

months = sorted(set(zhvi['month_str']))

tmp_selectionslider = ipw.widgets.SelectionSlider(
    options=months,
    value='2017-12',
    description='Month:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout=ipw.Layout(width='50%')
)

# Arrange widgets
tmp_w = [tmp_dropdown, tmp_selectionslider]
tmp_w_box = ipw.HBox(tmp_w)

ipd.display(tmp_w_box)

We can access each widgets value directly.

In [23]:
print('Dropdown: ', tmp_dropdown.value)
print('Slider: ', tmp_selectionslider.value)

Dropdown:  Stronghold
Slider:  2017-12


This means we can feed these arguments into functions.

In [24]:
zhvi_map = plot_zhvi(month=tmp_selectionslider.value, nhb=tmp_dropdown.value)
show_iframe(zhvi_map[0], iwidth=850, iheight=1050)

<iframe src=../figs/zhvi_dc.html width=850 height=1050></iframe>


Now we have GUI tools that can help set common parameters. If we build a layout with all the components, we have a standalone exploratory tool.

In [26]:
# Define location of output file
outf = fig_dir + 'zhvi_explore.html'
# bp.output_file(outf)

# Generate widgets
dropdown = ipw.widgets.Select(
    options=nhb_names,
    value='Stronghold',
    rows=8,
    description='Neighborhood:',
    disabled=False,
    style = {'description_width': 'initial'}
)

months = sorted(set(zhvi['month_str']))

selectionslider = ipw.widgets.SelectionSlider(
    options=months,
    value='2017-12',
    description='Month:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    layout=ipw.Layout(width='50%')
)

# Arrange widgets
selection_w = [dropdown, selectionslider]
selection_w_box = ipw.HBox(selection_w)

# Generate plots
map_plot = plot_zhvi(month=selectionslider.value, nhb=dropdown.value, save=False)[1]
ts_plot = plot_zhvi_ts(month=selectionslider.value, nhb=dropdown.value, save=False)[1]

# Define callback to refresh plots when widget values change
def refresh_plots(change):
    ipd.clear_output(wait=True)
    map_plot = plot_zhvi(month=selectionslider.value, nhb=dropdown.value, save=False)[1]
    ts_plot = plot_zhvi_ts(month=selectionslider.value, nhb=dropdown.value, save=False)[1]
    zhvi_explore = bl.layout([[map_plot], [ts_plot]])
    ipd.display(selection_w_box)
    bp.show(zhvi_explore, new='tab')
dd_plots = dropdown.observe(refresh_plots)
sel_plots = selectionslider.observe(refresh_plots)

# Combine time series plot with selectors
right_col = ipw.VBox([selection_w_box, ts_plot])

# Make the map a standalone
left_col = ipw.VBox([map_plot])

# Combine all components
zhvi_explore = ipw.HBox([left_col, right_col])
zhvi_explore = bl.layout([[map_plot], [ts_plot]])
bp.save(zhvi_explore, outf)

ipd.display(selection_w_box)
bp.show(zhvi_explore, notebook_url='localhost:8889')
show_iframe(fig_dir + 'zhvi_explore.html', iwidth=850, iheight=1450)

TraitError: Element of the 'children' trait of a VBox instance must be a Widget, but a value of Figure(id='0d941678-be50-4d22-ad3c-973fdc97e94c', ...) <class 'bokeh.plotting.figure.Figure'> was specified.

In [27]:
import bokeh.layouts as bl
import bokeh.io as bio

bio.output_notebook()

Loading BokehJS ...

In [ ]:
help(bp.show)